In [0]:
!git clone https://github.com/NTX-McGill/NeuroTechX-McGill-2019.git

Cloning into 'NeuroTechX-McGill-2019'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 4086 (delta 21), reused 38 (delta 13), pack-reused 4026
Receiving objects: 100% (4086/4086), 326.19 MiB | 13.20 MiB/s, done.
Resolving deltas: 100% (2184/2184), done.
Checking out files: 100% (478/478), done.


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
cd /content/drive/My\ Drive/NeuroTechX-McGill-2019

/content/drive/My Drive/NeuroTechX-McGill-2019


In [15]:
cd offline/ML

/content/drive/My Drive/NeuroTechX-McGill-2019/offline/ML


In [0]:
import pandas as pd
import numpy as np

#from metadata import MARKER_DATA, DATA_COLUMNS, LABELS, ALL_FILES


def merge_dols(dol1, dol2):
    keys = set(dol1).union(dol2)
    no = []
    return dict((k, dol1.get(k, no) + dol2.get(k, no)) for k in keys)

def merge_all_dols(arr):
    all_data = {'Right': [], 'Left': [], 'Rest': []}
    for dol in arr:
        all_data = merge_dols(all_data, dol)
    return all_data

def load_openbci_raw(path):
    data = np.loadtxt(path,
                      delimiter=',',
                      skiprows=7,
                      usecols=DATA_COLUMNS)
    eeg = data[:, :-1]
    timestamps = data[:, -1]
    return eeg, timestamps


def load_data(csv):
    print("loading " + csv)
    data = {label: [] for label in LABELS}
    df = pd.read_csv('../' + csv)
    path_arr = csv.split('/')
    folder, fname = path_arr[:-1], path_arr[-1]
    data_path = "/".join(['..'] + folder + [MARKER_DATA[fname]])
    eeg, timestamps = load_openbci_raw(data_path)
    prev = 0
    prev_direction = df['Direction'][prev]
    for idx, el in enumerate(df['Direction']):
        if el != prev_direction or idx == len(df.index) - 1:
            start = df['Time'][prev]
            end = df['Time'][idx]
            indices = np.where(
                np.logical_and(
                    timestamps >= start,
                    timestamps <= end))
            trial = eeg[indices]
            data[prev_direction].append(trial)
            prev = idx
            prev_direction = el
    return data


def load_dataset(csv_set):
    dataset = {label: [] for label in LABELS}
    for csv in csv_set:
        try:
            data = load_data(csv)
            dataset = merge_dols(dataset, data)
        except Exception as e: print(e)
    return dataset


def load_all():
    return {fname: load_data(fname) for fname in ALL_FILES}


In [0]:
import numpy as np
import numpy.fft as fft
from scipy import signal
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

#from metadata import SAMPLING_FREQ


def filter_signal(arr, lowcut, highcut, order, notch=True):
    if notch:
        arr = notch_mains_interference(arr)
    nyq = 0.5 * SAMPLING_FREQ
    b, a = signal.butter(1, [lowcut / nyq, highcut / nyq], btype='band')
    for i in range(0, order):
        arr = signal.lfilter(b, a, arr, axis=0)
    return arr


def notch_mains_interference(data):
    notch_freq_Hz = np.array([60.0])  # main + harmonic frequencies
    for freq_Hz in np.nditer(notch_freq_Hz):  # loop over each target freq
        bp_stop_Hz = freq_Hz + 3.0 * np.array([-1, 1])  # set the stop band
        b, a = signal.butter(3, bp_stop_Hz / (SAMPLING_FREQ / 2.0), 'bandstop')
        arr = signal.lfilter(b, a, data, axis=0)
        print("Notch filter removing: " +
              str(bp_stop_Hz[0]) +
              "-" +
              str(bp_stop_Hz[1]) +
              " Hz")
    return arr


def get_artifact_indices(ch):
    start_indices = [0]
    i = 0
    while i < len(ch):
        if ch[i] > 100:
            start_indices.append(i)
            i += 500
        i += 1
    return start_indices


def get_psd(ch, fs_Hz, shift=0.1):
    NFFT = fs_Hz * 2
    overlap = NFFT - int(shift * fs_Hz)
    psd, freqs = mlab.psd(np.squeeze(ch),
                          NFFT=NFFT,
                          window=mlab.window_hanning,
                          Fs=fs_Hz,
                          noverlap=overlap
                          )  # returns PSD power per Hz
    # convert the units of the spectral data
    spec_PSDperBin = spec_PSDperHz * fs_Hz / float(NFFT)
    return psd, freqs  # dB re: 1 uV


def get_spectral_content(ch, fs_Hz, shift=0.1):
    NFFT = fs_Hz * 2
    overlap = NFFT - int(shift * fs_Hz)
    spec_PSDperHz, spec_freqs, spec_t = mlab.specgram(np.squeeze(ch),
                                                      NFFT=NFFT,
                                                      window=mlab.window_hanning,
                                                      Fs=fs_Hz,
                                                      noverlap=overlap
                                                      )  # returns PSD power per Hz
    # convert the units of the spectral data
    spec_PSDperBin = spec_PSDperHz * fs_Hz / float(NFFT)
    return spec_t, spec_freqs, spec_PSDperBin  # dB re: 1 uV


def plot_specgram(spec_freqs, spec_PSDperBin, title, shift, i=1):
    f_lim_Hz = [0, 20]   # frequency limits for plotting
    # plt.figure(figsize=(10,5))
    spec_t = [idx * .1 for idx in range(len(spec_PSDperBin[0]))]
    plt.subplot(3, 1, i)
    plt.title(title)
    plt.pcolor(spec_t, spec_freqs, 10 *
               np.log10(spec_PSDperBin))  # dB re: 1 uV
    plt.clim([-25, 26])
    plt.xlim(spec_t[0], spec_t[-1] + 1)
    plt.ylim(f_lim_Hz)
    plt.xlabel('Time (sec)')
    plt.ylabel('Frequency (Hz)')
    plt.subplots_adjust(hspace=1)


def resize_min(specgram, i=1):
    min_length = min([len(el[0]) for el in specgram])
    specgram = np.array([el[:, :min_length] for el in specgram])
    return specgram


def resize_max(specgram, fillval=np.nan):
    max_length = max([len(el[0]) for el in specgram])
    return np.array([pad_block(el, max_length, fillval) for el in specgram])


def pad_block(block, max_length, fillval):
    padding = np.full([len(block), max_length - (len(block[0]))], fillval)
    return np.hstack((block, padding))


In [0]:
ELECTRODE_C3 = 0
ELECTRODE_C1 = 1
ELECTRODE_C2 = -2
ELECTRODE_C4 = -1

SAMPLING_FREQ = 250

DATA_COLUMNS = (1, 2, 3, 4, 5, 6, 7, 8, 13)
LABELS = ['Left', 'Right', 'Rest']

MARKER_DATA = {"1_011_Rest20LeftRight20_MI-2019-3-24-16-25-41.csv": '011_1to3_OpenBCI-RAW-2019-03-24_16-21-59.txt',
               "2_011_Rest20LeftRight20_MI-2019-3-24-16-38-10.csv": '011_1to3_OpenBCI-RAW-2019-03-24_16-21-59.txt',
               "3_011_Rest20LeftRight10_MI-2019-3-24-16-49-23.csv": '011_1to3_OpenBCI-RAW-2019-03-24_16-21-59.txt',
               "4_011_Rest20LeftRight10_MI-2019-3-24-16-57-8.csv": '011_4to6_OpenBCI-RAW-2019-03-24_16-54-15.txt',
               "5_011_Rest20LeftRight20_MI-2019-3-24-17-3-17.csv": '011_4to6_OpenBCI-RAW-2019-03-24_16-54-15.txt',
          
               }
          
FILES_BY_SUBJECT = [         
             ["data/March24_011/1_011_Rest20LeftRight20_MI-2019-3-24-16-25-41.csv",  # 011
              "data/March24_011/2_011_Rest20LeftRight20_MI-2019-3-24-16-38-10.csv",
              "data/March24_011/3_011_Rest20LeftRight10_MI-2019-3-24-16-49-23.csv",
              "data/March24_011/4_011_Rest20LeftRight10_MI-2019-3-24-16-57-8.csv",
              "data/March24_011/5_011_Rest20LeftRight20_MI-2019-3-24-17-3-17.csv",
              ]           
             ]
ALL_FILES = [name for sublist in FILES_BY_SUBJECT for name in sublist]


In [47]:
import warnings
from sklearn.utils import shuffle
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import seaborn as sn
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import numpy.fft as fft
import numpy as np
from scipy import stats
from scipy import signal

import sys
sys.path.append('../utils')
#from metadata import MARKER_DATA, LABELS, FILES_BY_SUBJECT, ALL_FILES, ELECTRODE_C3, ELECTRODE_C4
#import file_utils

warnings.filterwarnings("ignore")

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax



def plot_specgram(spec_freqs, spec_PSDperBin, title, shift, i=1):
    f_lim_Hz = [0, 20]   # frequency limits for plotting
    spec_t = [idx*.1 for idx in range(len(spec_PSDperBin[0]))]
    plt.subplot(3, 1, i)
    plt.title(title)
    plt.pcolor(spec_t, spec_freqs, 10*np.log10(spec_PSDperBin))  # dB re: 1 uV
    plt.clim([-25, 26])
    plt.xlim(spec_t[0], spec_t[-1]+1)
    plt.ylim(f_lim_Hz)
    plt.xlabel('Time (sec)')
    plt.ylabel('Frequency (Hz)')
    plt.subplots_adjust(hspace=1)


def epoch_data(data, window_length, shift, maxlen=2560):
    arr = []
    start = 0
    i = start
    maxlen = min(len(data), maxlen)
    while i + window_length < start + maxlen:
        arr.append(data[i:i+window_length])
        i += shift
    return np.array(arr)


def extract_features(all_data, window_s, shift, plot_psd=False, separate_trials=False, scale_by=None):
    all_psds = {label: [] for label in LABELS}
    all_features = {label: [] for label in LABELS}

    idx = 1
    for direction, data in all_data.items():
        for trial in data:
            epochs = epoch_data(trial, int(250 * window_s), int(shift*250))    # shape n x 500 x 2
            trial_features = []
            for epoch in epochs:
                print('epoch_shape: ',epoch.shape)
                features, freqs, psds_per_channel = get_features(epoch.T, scale_by=scale_by)
                print('freqs:',freqs)
                psd_c3, psd_c4 = psds_per_channel[0] , psds_per_channel[-1]
                all_psds[direction].append([psd_c3, psd_c4])
                trial_features.append(features)
                
                # Sanity check: plot the psd
                if plot_psd:
                    plt.figure("psd")
                    plt.subplot(3, 2, idx)
                    plt.plot(freqs, psd_c3)
                    plt.ylim([0, 25])
                    plt.xlim([6, 20])
                    plt.subplot(3, 2, idx+1)
                    plt.plot(freqs, psd_c4)
                    plt.ylim([0, 25])
                    plt.xlim([6, 20])
            if trial_features:
                if separate_trials:
                    all_features[direction].append(np.array(trial_features))
                else:
                    all_features[direction].extend(trial_features)
        idx += 2
    return all_psds, all_features, freqs


def to_feature_vec(all_features, rest=False):
    feature_arr = []
    for direction, features in all_features.items():
        features = np.array(features)
        arr = np.hstack((features, np.full([features.shape[0], 1], LABELS.index(direction))))
        feature_arr.append(arr)
    if not rest or not len(features):
        feature_arr = feature_arr[:-1]
    return np.vstack(feature_arr)


def normalize(features_dict):
    """ Divide features by mean per channel """
    all_features = to_feature_vec(features_dict, rest=True)
    av = list(np.mean(all_features, axis=0))
    mean_coeff = np.array([el/sum(av[:-1]) for el in av[:-1]])
    for direction, features in features_dict.items():
        features = [np.divide(example, mean_coeff) for example in features]
        features_dict[direction] = features
    
def running_mean(x, N):
   cumsum = np.cumsum(np.insert(x, 0, 0)) 
   return (cumsum[N:] - cumsum[:-N]) / N


def evaluate_models(X, Y, X_test, Y_test, models):
    """ Evaluate test accuracy of all models in a list of models
    
    Args:
        X : array of features (train)
        Y : array of labels (train)
        X_test : array of features (test)
        Y_test : array of labels (test)
        models : list of (name, model) tuples
    
    Returns:
        val_results : array of accuracies, shape num_models
    """
    test_results = []
    for name, model in models:
        model.fit(X, Y)
        score = model.score(X_test, Y_test)
        test_results.append(score)
        msg = "%s: %f" % (name, score)
        print(msg)
    return np.array(test_results)


def evaluate_models_crossval(X, Y, models, scoring, random_state, n_splits=10):
    """ Evaluate cross-validation accuracy of all models in a list of models
    
    Args:
        X : array of features
        Y : array of labels
        models : list of (name, model) tuples
        scoring : string, scoring metric
        random_state : seed to use for shuffling
        n_splits : number of folds
    
    Returns:
        val_results : array of accuracies, shape num_models x n_splits
    """
    val_results = []
    for name, model in models:
        kfold = model_selection.KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        cv_results = model_selection.cross_val_score(model, X_test, Y_test, cv=kfold, scoring=scoring)
        val_results.append(cv_results)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
    return np.array(val_results)


def get_features(arr, channels=[ELECTRODE_C3, ELECTRODE_C4], scale_by=None):
    """ Get features from single window of EEG data
    
    Args:
        arr : data of shape num_channels x timepoints
    
    Returns:
        features : array with feature values
        freqs : array of frequencies in Hz
        psds_per_channel : array with full psd spectrum, shape num_channels x num_freqs
    """

    psds_per_channel = []
    nfft = 500
    if arr.shape[-1] < 500:
        nfft = 250
    print(arr[channels])    
    for ch in arr[channels]:        
        #freqs,psd = signal.periodogram(np.squeeze(ch),fs=250, nfft=500, detrend='constant')
        psd, freqs, = mlab.psd(np.squeeze(ch),NFFT=nfft,Fs=250)
        print('freqs: ',freqs)
        psds_per_channel.append(psd)
    psds_per_channel = np.array(psds_per_channel)
    mu_indices = np.where(np.logical_and(freqs >= 10, freqs <= 12))
    
    #features = np.amax(psds_per_channel[:,mu_indices], axis=-1).flatten()   # max of 10-12hz as feature
    features = np.mean(psds_per_channel[:, mu_indices], axis=-1).flatten()   # mean of 10-12hz as feature
    if scale_by:
        scale_indices = np.where(np.logical_and(freqs >= scale_by[0], freqs <= scale_by[-1]))
        scales = np.mean(psds_per_channel[:,scale_indices],axis=-1).flatten()
        temp.append(scales)
        features = np.divide(features, scales)
    #features = np.array([features[:2].mean(), features[2:].mean()])
    # features = psds_per_channel[:,mu_indices].flatten()                     # all of 10-12hz as feature
    return features, freqs, psds_per_channel


if __name__ == "__main__":
    shift = 0.25
    plot_psd = False
    tmin, tmax = 0, 0
    window_lengths = [1, 2, 4]  # window lengths in seconds
    normalize_spectra = True
    run_pca = False
    scale_by = None
    
    # Load data
    dataset = load_all()
    subjects = [i for i in range(len(FILES_BY_SUBJECT))]             # index of the test files we want to use
    
    all_val_results = []
    all_test_results = []

    # Test options and evaluation metric
    scoring = 'accuracy'
    validation = True
    test = False
    seed = 7
    models = []
    models.append(('LR', LogisticRegression(solver='lbfgs')))
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('CART', DecisionTreeClassifier()))
    models.append(('NB', GaussianNB(var_smoothing=0.001)))
    models.append(('SVM', SVC(gamma='scale')))
    
    # Perform leave-one-subject-out cross-validation for each subject
    # Delivers accuracy by subject, then by window size, then by session, then by model
    for subj in subjects:
        test_csvs = FILES_BY_SUBJECT[subj]
        train_csvs = [el for el in ALL_FILES if el not in test_csvs]
        train_data = merge_all_dols([dataset[csv] for csv in train_csvs])
        
        # Print subject name
        print(test_csvs[0].split('/')[1])
        print('Puta prueba')
        window_val_results = []
        window_test_results = []
        print(type(train_data))
        print(window_s)
        
        for window_s in window_lengths:
            train_psds, train_features, freqs = extract_features(train_data, window_s, shift, plot_psd, scale_by=scale_by)
            data = to_feature_vec(train_features, rest=False)
            
            # X, Y for training
            # For testing: X_test, Y_test
            X = data[:, :-1]
            Y = data[:, -1]
            X, Y = shuffle(X, Y, random_state=seed)
            if run_pca:
                pca = PCA(n_components=2, svd_solver='full')
                pca.fit(X)
                X = pca.transform(X)
            
            subj_val_results = []
            subj_test_results = []
            for csv in test_csvs:
                test_data = dataset[csv]
                _, test_features, _ = extract_features(test_data, window_s, shift, plot_psd, scale_by=scale_by)
                if normalize_spectra:
                    normalize(test_features)
                test_data = to_feature_vec(test_features)
                X_test = test_data[:, :-1]
                Y_test = test_data[:, -1]
                if run_pca:
                    X_test = pca.transform(X_test)
                
                if validation:
                    print("VALIDATION")
                    val_results = evaluate_models_crossval(X_test, Y_test, models, scoring, seed, n_splits=10)
                    print("average accuracy: " + "{:2.1f}".format(val_results.mean() * 100))
                    subj_val_results.append(val_results.mean() * 100)
                    
                if test:
                    print("TEST")
                    test_results = evaluate_models(X, Y, X_test, Y_test, models)
                    print("average accuracy: {:2.1f}".format(test_results.mean() * 100))
                    subj_test_results.append(test_results.mean() * 100)
            window_val_results.append([np.array(subj_val_results).mean(), stats.sem(np.array(subj_val_results))])
            window_test_results.append([np.array(subj_test_results).mean(),stats.sem(np.array(subj_test_results))])
        all_val_results.append(window_val_results)
        all_test_results.append(window_test_results)
        
    print(np.array(all_test_results).mean())


loading data/March24_011/1_011_Rest20LeftRight20_MI-2019-3-24-16-25-41.csv
loading data/March24_011/2_011_Rest20LeftRight20_MI-2019-3-24-16-38-10.csv
loading data/March24_011/3_011_Rest20LeftRight10_MI-2019-3-24-16-49-23.csv
loading data/March24_011/4_011_Rest20LeftRight10_MI-2019-3-24-16-57-8.csv
loading data/March24_011/5_011_Rest20LeftRight20_MI-2019-3-24-17-3-17.csv
March24_011
Puta prueba
<class 'dict'>
1


UnboundLocalError: ignored

In [48]:
train_data



{'Left': [], 'Rest': [], 'Right': []}

In [0]:
MARKER_DATA

{'10_008-2019-3-22-15-8-55.csv': '10_008_OpenBCI-RAW-2019-03-22_15-07-58.txt',
 '4_RestLeftRight_MI_5s.csv': '4_RestLeftRight_5s_MI_OpenBCI-RAW-2019-03-17_16-32-53.txt',
 '5_RestLeftRight_MI_10s.csv': '5_RestLeftRight_10s_OpenBCI-RAW-2019-03-17_16-37-32.txt'}

In [0]:
matplotlib.pyplot.psd(x, NFFT=None, Fs=None, Fc=None, detrend=None, window=None, noverlap=None, pad_to=None, sides=None, scale_by_freq=None, return_line=None, *, data=None, **kwargs)

In [40]:
psd, freqs = mlab.psd(np.squeeze([8,3]),NFFT=500,Fs=250)
print(freqs)

[  0.    0.5   1.    1.5   2.    2.5   3.    3.5   4.    4.5   5.    5.5
   6.    6.5   7.    7.5   8.    8.5   9.    9.5  10.   10.5  11.   11.5
  12.   12.5  13.   13.5  14.   14.5  15.   15.5  16.   16.5  17.   17.5
  18.   18.5  19.   19.5  20.   20.5  21.   21.5  22.   22.5  23.   23.5
  24.   24.5  25.   25.5  26.   26.5  27.   27.5  28.   28.5  29.   29.5
  30.   30.5  31.   31.5  32.   32.5  33.   33.5  34.   34.5  35.   35.5
  36.   36.5  37.   37.5  38.   38.5  39.   39.5  40.   40.5  41.   41.5
  42.   42.5  43.   43.5  44.   44.5  45.   45.5  46.   46.5  47.   47.5
  48.   48.5  49.   49.5  50.   50.5  51.   51.5  52.   52.5  53.   53.5
  54.   54.5  55.   55.5  56.   56.5  57.   57.5  58.   58.5  59.   59.5
  60.   60.5  61.   61.5  62.   62.5  63.   63.5  64.   64.5  65.   65.5
  66.   66.5  67.   67.5  68.   68.5  69.   69.5  70.   70.5  71.   71.5
  72.   72.5  73.   73.5  74.   74.5  75.   75.5  76.   76.5  77.   77.5
  78.   78.5  79.   79.5  80.   80.5  81.   81.5  8

In [34]:
freqs,psd = signal.periodogram(np.squeeze([0.787,0.453,0.876]),fs=250, nfft=500, detrend='constant')
print(freqs, psd)

[  0.    0.5   1.    1.5   2.    2.5   3.    3.5   4.    4.5   5.    5.5
   6.    6.5   7.    7.5   8.    8.5   9.    9.5  10.   10.5  11.   11.5
  12.   12.5  13.   13.5  14.   14.5  15.   15.5  16.   16.5  17.   17.5
  18.   18.5  19.   19.5  20.   20.5  21.   21.5  22.   22.5  23.   23.5
  24.   24.5  25.   25.5  26.   26.5  27.   27.5  28.   28.5  29.   29.5
  30.   30.5  31.   31.5  32.   32.5  33.   33.5  34.   34.5  35.   35.5
  36.   36.5  37.   37.5  38.   38.5  39.   39.5  40.   40.5  41.   41.5
  42.   42.5  43.   43.5  44.   44.5  45.   45.5  46.   46.5  47.   47.5
  48.   48.5  49.   49.5  50.   50.5  51.   51.5  52.   52.5  53.   53.5
  54.   54.5  55.   55.5  56.   56.5  57.   57.5  58.   58.5  59.   59.5
  60.   60.5  61.   61.5  62.   62.5  63.   63.5  64.   64.5  65.   65.5
  66.   66.5  67.   67.5  68.   68.5  69.   69.5  70.   70.5  71.   71.5
  72.   72.5  73.   73.5  74.   74.5  75.   75.5  76.   76.5  77.   77.5
  78.   78.5  79.   79.5  80.   80.5  81.   81.5  8